# Grab data

In [10]:
import pandas as pd
import lib.cleaners as c
import numpy as np

dataset = pd.read_json('dataset/resume_output_waiter-canada.json', lines=True)
dataset.head(10)
print('Dataset (maybe) duplicated:', len(dataset))

dataset = dataset.drop_duplicates(subset=['id'])
print('Dataset non-duplicated:', len(dataset))

Dataset (maybe) duplicated: 14979
Dataset non-duplicated: 14958


# Expanding education data

Expand education data into multiple rows

In [11]:
edu_df = c.expand_to_multi_rows(dataset, 'schools', 'school')
edu_df = edu_df[['id', 'school']]
edu_df.head(10)

,id,school
0,445795a780dab6f0,{'degree': 'High school diploma in Fly attenda...
1,2244a003710b9a51,{'degree': 'Advanced Diploma in Computer Engin...
2,6e38f142ee50ad40,{'degree': 'in Global Business & Digital Arts'...
3,6e38f142ee50ad40,"{'degree': '', 'school_name': 'University of W..."
4,6e38f142ee50ad40,"{'degree': '', 'school_name': 'Vincent Massey ..."
5,dfbc2ef2cabe7350,"{'degree': 'BA in Finance', 'school_name': 'Un..."
6,3818623ba1d16bcf,"{'degree': 'HIGH SCHOOL DIPLOMA', 'school_name..."
7,e0e3e777776d80b2,"{'degree': 'in Computer Science', 'school_name..."
8,e0e3e777776d80b2,"{'degree': '', 'school_name': 'Robert Service ..."
9,74ff36218d04583d,"{'degree': 'in Supply Chain', 'school_name': '..."


Make dataframe out of the education dictionary

In [12]:
degree_df = c.create_new_df_dict_col(edu_df, 'school')

Delete NaN degrees and replace empty strings to NaN

In [16]:
degree_df = degree_df.replace('', np.nan)
degree_df = degree_df.dropna(subset=['degree'])
degree_df.head(10)

,degree,school_name,start_date,end_date,degree_year_time
0,High school diploma in Fly attendant,Hostess and steward academy,2019-06-01,2027-06-01,8.000164
1,Advanced Diploma in Computer Engineering Techn...,Humber College,2018-09-01,2021-05-01,2.663984
2,in Global Business & Digital Arts,University of Waterloo,NaT,NaT,NaN
5,BA in Finance,University of Guelph,2018-09-01,2021-04-01,2.581846
6,HIGH SCHOOL DIPLOMA,NORTH NOVA EDUCATION CENTER,2018-06-01,NaT,NaN
7,in Computer Science,York University,2018-09-01,NaT,NaN
9,in Supply Chain,George Brown College,2019-01-01,2023-01-01,4.000082
11,Master of Engineering in Industrial Engineering,Concordia University starting,NaT,NaT,NaN
15,in Automotive Collision Repair Diploma,Vancouver community college,2018-09-01,2020-04-01,1.582510
16,Secondary School,NaN,NaT,NaT,NaN


Simplifying degree data (it modifies the given DataFrame)

In [17]:
c.simplify_education_information_(degree_df, 'start_date', 'end_date')
simplified_degree_df = degree_df[['degree', 'degree_year_time']]
simplified_degree_df.head(10)

ValueError: ('Unknown string format:', 'Present')

Combine into original DF with ID

In [18]:
edu_df = edu_df[['id']]
expanded_resume_degree = pd.merge(edu_df, simplified_degree_df, left_index=True, right_index=True)
expanded_resume_degree.head(10)

,id,degree,degree_year_time
0,445795a780dab6f0,High school diploma in Fly attendant,8.000164
1,2244a003710b9a51,Advanced Diploma in Computer Engineering Techn...,2.663984
2,6e38f142ee50ad40,in Global Business & Digital Arts,NaN
5,dfbc2ef2cabe7350,BA in Finance,2.581846
6,3818623ba1d16bcf,HIGH SCHOOL DIPLOMA,NaN
7,e0e3e777776d80b2,in Computer Science,NaN
9,74ff36218d04583d,in Supply Chain,4.000082
11,9585295f9745285a,Master of Engineering in Industrial Engineering,NaN
15,24a0a8d680b41793,in Automotive Collision Repair Diploma,1.582510
16,5f2425766c95966e,Secondary School,NaN


# Expanding skills data

Expand skills data to multiple rows

In [8]:
skills_df = c.expand_to_multi_rows(dataset, 'skills', 'skill_dict')
skills_df.head(10)

,additional,id,jobs,schools,skills,summary,skill_dict
0,"[Skills, -Good writing and reading abilities.,...",0e84aa0202047520,"[{'title': 'Line Cook and Prep Cook', 'company...","[{'degree': 'Certificate in French Immersion',...","[{'skill': 'COOKING', 'experience': '4 years'}...","[Well organized, focused individual who strive...","{'skill': 'COOKING', 'experience': '4 years'}"
1,"[Skills, -Good writing and reading abilities.,...",0e84aa0202047520,"[{'title': 'Line Cook and Prep Cook', 'company...","[{'degree': 'Certificate in French Immersion',...","[{'skill': 'COOKING', 'experience': '4 years'}...","[Well organized, focused individual who strive...","{'skill': 'HAND TOOLS', 'experience': 'Less th..."
2,"[Skills, -Good writing and reading abilities.,...",0e84aa0202047520,"[{'title': 'Line Cook and Prep Cook', 'company...","[{'degree': 'Certificate in French Immersion',...","[{'skill': 'COOKING', 'experience': '4 years'}...","[Well organized, focused individual who strive...","{'skill': 'WHMIS', 'experience': 'Less than 1 ..."
3,[Currently looking for full time job (prefere ...,708b408f8988a22e,"[{'title': 'Dishwasher/Line Cook', 'company': ...",[{'degree': 'Master's degree in Mechanical eng...,"[{'skill': 'Microsoft Office', 'experience': '...",[],"{'skill': 'Microsoft Office', 'experience': '9..."
4,[Currently looking for full time job (prefere ...,708b408f8988a22e,"[{'title': 'Dishwasher/Line Cook', 'company': ...",[{'degree': 'Master's degree in Mechanical eng...,"[{'skill': 'Microsoft Office', 'experience': '...",[],"{'skill': 'Inventor', 'experience': '8 years'}"
5,[Currently looking for full time job (prefere ...,708b408f8988a22e,"[{'title': 'Dishwasher/Line Cook', 'company': ...",[{'degree': 'Master's degree in Mechanical eng...,"[{'skill': 'Microsoft Office', 'experience': '...",[],"{'skill': 'Cad', 'experience': '8 years'}"
6,[Currently looking for full time job (prefere ...,708b408f8988a22e,"[{'title': 'Dishwasher/Line Cook', 'company': ...",[{'degree': 'Master's degree in Mechanical eng...,"[{'skill': 'Microsoft Office', 'experience': '...",[],"{'skill': 'Creo Parametric', 'experience': '2 ..."
7,"[Skills, ● High energy attitude, ● Sales exper...",c7675c0b793f34a0,"[{'title': 'Manager', 'company': 'Pizza Hut', ...","[{'degree': 'Certificate', 'school_name': 'Geo...","[{'skill': 'CUSTOMER RELATIONS', 'experience':...",[Looking for part-time position. Goal focused ...,"{'skill': 'CUSTOMER RELATIONS', 'experience': ..."
8,"[Skills, ● High energy attitude, ● Sales exper...",c7675c0b793f34a0,"[{'title': 'Manager', 'company': 'Pizza Hut', ...","[{'degree': 'Certificate', 'school_name': 'Geo...","[{'skill': 'CUSTOMER RELATIONS', 'experience':...",[Looking for part-time position. Goal focused ...,"{'skill': 'CUSTOMER SUPPORT', 'experience': 'L..."
9,"[Skills, ● High energy attitude, ● Sales exper...",c7675c0b793f34a0,"[{'title': 'Manager', 'company': 'Pizza Hut', ...","[{'degree': 'Certificate', 'school_name': 'Geo...","[{'skill': 'CUSTOMER RELATIONS', 'experience':...",[Looking for part-time position. Goal focused ...,"{'skill': 'PROBLEM SOLVING', 'experience': 'Le..."


Create a DF using the dictionary of skills

In [9]:
expanded_skill_df = c.create_new_df_dict_col(skills_df, 'skill_dict')
expanded_skill_df.head(10)

,skill,experience
0,COOKING,4 years
1,HAND TOOLS,Less than 1 year
2,WHMIS,Less than 1 year
3,Microsoft Office,9 years
4,Inventor,8 years
5,Cad,8 years
6,Creo Parametric,2 years
7,CUSTOMER RELATIONS,Less than 1 year
8,CUSTOMER SUPPORT,Less than 1 year
9,PROBLEM SOLVING,Less than 1 year


Simplify skills information

In [10]:
c.simplify_skills_information_(expanded_skill_df, 'experience')
expanded_skill_df.head(10)

,skill,experience
0,COOKING,4
1,HAND TOOLS,1
2,WHMIS,1
3,Microsoft Office,9
4,Inventor,8
5,Cad,8
6,Creo Parametric,2
7,CUSTOMER RELATIONS,1
8,CUSTOMER SUPPORT,1
9,PROBLEM SOLVING,1


Combine into original DF with ID

In [17]:
skills_df = skills_df[['id']]
expanded_resume_skills = pd.merge(skills_df, expanded_skill_df, left_index=True, right_index=True)
expanded_resume_skills.head(10)

,id,skill,experience
0,0e84aa0202047520,COOKING,4
1,0e84aa0202047520,HAND TOOLS,1
2,0e84aa0202047520,WHMIS,1
3,708b408f8988a22e,Microsoft Office,9
4,708b408f8988a22e,Inventor,8
5,708b408f8988a22e,Cad,8
6,708b408f8988a22e,Creo Parametric,2
7,c7675c0b793f34a0,CUSTOMER RELATIONS,1
8,c7675c0b793f34a0,CUSTOMER SUPPORT,1
9,c7675c0b793f34a0,PROBLEM SOLVING,1


## Expanding jobs data

Expand to multiple rows for jobs data

In [3]:
jobs_df = c.expand_to_multi_rows(dataset, 'jobs', 'job')
jobs_df = jobs_df[['id', 'job']]
jobs_df.head(10)

,id,job
0,54a3bd53808e6764,"{'title': 'Bus Person', 'company': 'Knight and Day', 'start_date': 'January 2019', 'end_date': 'Present', 'details': []}"
1,54a3bd53808e6764,"{'title': 'Stacker', 'company': 'Dollarama', 'start_date': 'March 2017', 'end_date': 'April 2017', 'details': ['My school sent me here for work experience']}"
2,54a3bd53808e6764,"{'title': 'Stacker', 'company': 'Toys 'R' Us', 'start_date': 'January 2017', 'end_date': 'February 2017', 'details': ['School sent me for work experience']}"
3,54a3bd53808e6764,"{'title': 'Cook', 'company': 'Little Caesars', 'start_date': '', 'end_date': '', 'details': []}"
4,f8dc5fa9fc39c387,"{'title': 'Sushi Cook, Pans Cook, Appetizer Cook, Forno Cook', 'company': 'JOEY Restaurants', 'start_date': 'June 2017', 'end_date': 'Present', 'details': ['I worked Dishwasher, Apps, Pans, Forno ..."
5,f8dc5fa9fc39c387,"{'title': '', 'company': 'Calgary Public Library School's Out All Day', 'start_date': 'October 2016', 'end_date': 'Present', 'details': []}"
6,f8dc5fa9fc39c387,"{'title': 'City Of Calgary Recreation Day Camp Volunteer', 'company': '', 'start_date': 'July 2016', 'end_date': 'Present', 'details': []}"
7,f8dc5fa9fc39c387,"{'title': 'Volunteer Of The Week', 'company': '', 'start_date': 'August 2015', 'end_date': 'August 2016', 'details': []}"
8,0e84aa0202047520,"{'title': 'Line Cook and Prep Cook', 'company': 'Cathedral Social Hall', 'start_date': 'January 2016', 'end_date': 'Present', 'details': ['Safe handling and preparation of food. Cooking, cleaning,..."
9,0e84aa0202047520,"{'title': 'Line Cook and Prep Cook', 'company': 'Cuisine Boutique LOV', 'start_date': 'January 2018', 'end_date': 'January 2019', 'details': ['Safe handling of food and main dishes on hot section,..."


Make dictionary keys as columns

In [4]:
expanded_job_df = c.create_new_df_dict_col(jobs_df, 'job')
expanded_job_df.head(10)

,title,company,start_date,end_date,details
0,Bus Person,Knight and Day,January 2019,Present,[]
1,Stacker,Dollarama,March 2017,April 2017,[My school sent me here for work experience]
2,Stacker,Toys 'R' Us,January 2017,February 2017,[School sent me for work experience]
3,Cook,Little Caesars,,,[]
4,"Sushi Cook, Pans Cook, Appetizer Cook, Forno Cook",JOEY Restaurants,June 2017,Present,"[I worked Dishwasher, Apps, Pans, Forno and Sushi.]"
5,,Calgary Public Library School's Out All Day,October 2016,Present,[]
6,City Of Calgary Recreation Day Camp Volunteer,,July 2016,Present,[]
7,Volunteer Of The Week,,August 2015,August 2016,[]
8,Line Cook and Prep Cook,Cathedral Social Hall,January 2016,Present,"[Safe handling and preparation of food. Cooking, cleaning, stocking, organization of kitchen, organization of walk-in cooler. Following routine restaurant guidelines for, line set-up, and restaura..."
9,Line Cook and Prep Cook,Cuisine Boutique LOV,January 2018,January 2019,"[Safe handling of food and main dishes on hot section, training of new employees, stocking, organization of kitchen, cleaning and line-leadership.]"


Simplify jobs data (with punctuations and stopwords removed)

In [5]:
c.simplify_jobs_information_(expanded_job_df, 'start_date', 'end_date', 'details')
expanded_job_df.head(10)
simplified_job_df = expanded_job_df[['title', 'job_duration', 'job_details']]
simplified_job_df.head(10)

,title,company,start_date,end_date,job_details,job_duration
0,Bus Person,Knight and Day,NaT,NaT,[],NaN
1,Stacker,Dollarama,2017-03-01,2017-04-01,"[school, sent, work, experience]",0.084875
2,Stacker,Toys 'R' Us,2017-01-01,2017-02-01,"[school, sent, work, experience]",0.084875
3,Cook,Little Caesars,NaT,NaT,[],NaN
4,"Sushi Cook, Pans Cook, Appetizer Cook, Forno Cook",JOEY Restaurants,NaT,NaT,"[worked, dishwasher, ,, apps, ,, pans, ,, forno, sushi, .]",NaN
5,,Calgary Public Library School's Out All Day,NaT,NaT,[],NaN
6,City Of Calgary Recreation Day Camp Volunteer,,NaT,NaT,[],NaN
7,Volunteer Of The Week,,2015-08-01,2016-08-01,[],1.002074
8,Line Cook and Prep Cook,Cathedral Social Hall,NaT,NaT,"[safe, handling, preparation, food, ., cooking, ,, cleaning, ,, stocking, ,, organization, kitchen, ,, organization, walkin, cooler, ., following, routine, restaurant, guidelines, line, setup, ,, ...",NaN
9,Line Cook and Prep Cook,Cuisine Boutique LOV,2018-01-01,2019-01-01,"[safe, handling, food, main, dishes, hot, section, ,, training, new, employees, ,, stocking, ,, organization, kitchen, ,, cleaning, lineleadership, .]",0.999336


Merge back to expanded data

In [11]:
jobs_df = jobs_df[['id']]
expanded_resume_jobs = pd.merge(jobs_df, simplified_job_df, left_index=True, right_index=True)
expanded_resume_jobs.head(10)

,id,title,job_duration,job_details
0,54a3bd53808e6764,Bus Person,NaN,[]
1,54a3bd53808e6764,Stacker,0.084875,"[school, sent, work, experience]"
2,54a3bd53808e6764,Stacker,0.084875,"[school, sent, work, experience]"
3,54a3bd53808e6764,Cook,NaN,[]
4,f8dc5fa9fc39c387,"Sushi Cook, Pans Cook, Appetizer Cook, Forno Cook",NaN,"[worked, dishwasher, apps, pans, forno, sushi]"
5,f8dc5fa9fc39c387,,NaN,[]
6,f8dc5fa9fc39c387,City Of Calgary Recreation Day Camp Volunteer,NaN,[]
7,f8dc5fa9fc39c387,Volunteer Of The Week,1.002074,[]
8,0e84aa0202047520,Line Cook and Prep Cook,NaN,"[safe, handling, preparation, food, cooking, c..."
9,0e84aa0202047520,Line Cook and Prep Cook,0.999336,"[safe, handling, food, main, dishes, hot, sect..."


# Clean summary data

In [2]:
pd.set_option('display.max_colwidth', 200)
cleaned_summary = c.tokenize_array_of_sentences_(dataset, 'summary')
cleaned_summary.head(10)

,additional,id,jobs,schools,skills,summary
0,"[SKILLS & ABILITIES:, • Young, Energetic and hard working., • Responsible and punctual., • Able to work independently and as a team player., • Adaptable and willing to learn new concepts., • Excel...",54a3bd53808e6764,"[{'title': 'Bus Person', 'company': 'Knight and Day', 'start_date': 'January 2019', 'end_date': 'Present', 'details': []}, {'title': 'Stacker', 'company': 'Dollarama', 'start_date': 'March 2017', ...","[{'degree': '', 'school_name': 'Queen Elizabeth secondary', 'start_date': 'January 2012', 'end_date': 'January 2017'}, {'degree': '', 'school_name': 'Kpu', 'start_date': 'September 2018', 'end_dat...",None,"[obtain, part-time, job, atmosphere, use, improve, customer, service, skills]"
1,"[SKILLS, - Reliability, - Leadership and management, - Research and information gathering skills, - Listening skills, - Creativity and innovation, - Ability to manage and organize info...",f8dc5fa9fc39c387,"[{'title': 'Sushi Cook, Pans Cook, Appetizer Cook, Forno Cook', 'company': 'JOEY Restaurants', 'start_date': 'June 2017', 'end_date': 'Present', 'details': ['I worked Dishwasher, Apps, Pans, Forno...","[{'degree': 'High School Diploma in Technical Theater', 'school_name': 'BISHOP CARROLL HIGH SCHOOL', 'start_date': 'September 2016', 'end_date': 'Present'}]",None,[]
2,"[Skills, -Good writing and reading abilities., -Able to speak and write both French and English., -Familiar with electronics and computer equipment., -Proficient in computer and internet programs....",0e84aa0202047520,"[{'title': 'Line Cook and Prep Cook', 'company': 'Cathedral Social Hall', 'start_date': 'January 2016', 'end_date': 'Present', 'details': ['Safe handling and preparation of food. Cooking, cleaning...","[{'degree': 'Certificate in French Immersion', 'school_name': 'Ecole Connaught Community School', 'start_date': 'January 2011', 'end_date': ''}]","[{'skill': 'COOKING', 'experience': '4 years'}, {'skill': 'HAND TOOLS', 'experience': 'Less than 1 year'}, {'skill': 'WHMIS', 'experience': 'Less than 1 year'}]","[well, organized, focused, individual, strives, enhance, workplace, enjoy, contributing, participating, kind, healthy, work, environment, prepared, offer, support, needed, adaptable, alert, attent..."
3,"[• Experienced hands-on worker; some construction experience, driving, • Responsible worker - very polite, • Accurate; pay attention to detail, • Good physical condition and eager to work, • Good ...",67b00f0985c8948a,"[{'title': 'washer & dishwasher', 'company': '', 'start_date': 'January 2016', 'end_date': 'January 2018', 'details': []}, {'title': '', 'company': 'Compass group Canada', 'start_date': 'January 2...","[{'degree': 'in Mechanics and German Language Studies', 'school_name': 'Hassi Messaoud High School', 'start_date': '', 'end_date': ''}]",None,"[driving, position]"
4,[Currently looking for part time work in the evenings 3-4 nights a week],89c76f9c1242c552,"[{'title': 'Cook', 'company': 'Sunridge Gardens', 'start_date': 'May 2014', 'end_date': 'January 2015', 'details': ['-Prepare and', 'cook', 'food for lunch/ dinner service', '-Accommodating di...","[{'degree': 'Professional cooks lvl 1 in Culinary arts', 'school_name': 'Collage', 'start_date': 'May 2013', 'end_date': 'April 2014'}, {'degree': 'High school diploma', 'school_name': 'Fleetwood...",None,[]
5,None,84ebc2bebda72365,"[{'title': 'Cashier/Cook', 'company': 'Jack Pirtles Chicken', 'start_date': 'January 2018', 'end_date': 'Present', 'details': ['• Receives food orders via t.v. monitor and prepares meal combinatio...","[{'degree': 'High School Diploma', 'school_name': 'Hamilton High School', 'start_date': 'May 1975', 'end_date': 'May 1979'}]",None,[]
6,None,017658f6f804297c,"[{'title': 'Casual Cook', 'company': 'The Bridge Youth & Family Services', 'start_date': 'April 2014', 'end_date': 'Present', 'details': ['preparation and service of food at grey road intensive tr...","[{'degree': 'apprenticehip in the field of cooking in culin

# Clean additional data

In [23]:
pd.set_option('display.max_colwidth', 200)
# just an example with not removing punctuation, in real thing we should remove
cleaned_additional = c.clean_array_of_sentences(dataset, 'additional', clean_punctuation=False)
cleaned_additional.head(50,)

0     [skills, &, abilities, :, , young, ,, energetic, hard, working, ., , responsible, punctual, ., , able, work, independently, team, player, ., , adaptable, willing, learn, new, concepts, ., , excell...
1     [skills, , reliability, , leadership, management, , research, information, gathering, skills, , listening, skills, , creativity, innovation, , ability, manage, organize, information, , sense, resp...
2     [skills, good, writing, reading, abilities, ., able, speak, write, french, english, ., familiar, electronics, computer, equipment, ., proficient, computer, internet, programs, ., able, focus, well...
3     [, experienced, handson, worker, ;, construction, experience, ,, driving, , responsible, worker, , polite, , accurate, ;, pay, attention, detail, , good, physical, condition, eager, work, , good, ...
4                                                                                                                                          [currently, looking, part, time, work

# Merge data together into one DF

In [28]:
dataset_ids = dataset['id']
merged_and_cleaned = pd.merge(dataset_ids, cleaned_summary, left_index=True, right_index=True, how='inner')
merged_and_cleaned = pd.merge(merged_and_cleaned, cleaned_additional, left_index=True, right_index=True, how='inner')

merged_and_cleaned.head(10)

,id,summary,additional
0,54a3bd53808e6764,"[obtain, parttime, job, atmosphere, use, improve, customer, service, skills]","[skills, &, abilities, :, , young, ,, energetic, hard, working, ., , responsible, punctual, ., , able, work, independently, team, player, ., , adaptable, willing, learn, new, concepts, ., , excell..."
1,f8dc5fa9fc39c387,[],"[skills, , reliability, , leadership, management, , research, information, gathering, skills, , listening, skills, , creativity, innovation, , ability, manage, organize, information, , sense, resp..."
2,0e84aa0202047520,"[well, organized, focused, individual, strives, enhance, workplace, enjoy, contributing, participating, kind, healthy, work, environment, prepared, offer, support, needed, adaptable, alert, attent...","[skills, good, writing, reading, abilities, ., able, speak, write, french, english, ., familiar, electronics, computer, equipment, ., proficient, computer, internet, programs, ., able, focus, well..."
3,67b00f0985c8948a,"[driving, position]","[, experienced, handson, worker, ;, construction, experience, ,, driving, , responsible, worker, , polite, , accurate, ;, pay, attention, detail, , good, physical, condition, eager, work, , good, ..."
4,89c76f9c1242c552,[],"[currently, looking, part, time, work, evenings, 34, nights, week]"
8,708b408f8988a22e,[],"[currently, looking, full, time, job, (, prefere, morning, shift, ), ., intrested, ,, please, contact, via, email, ermis.j, @, seznam.cz.my, visa, wh, expirate, august, 2019., thank, looking, forw..."
9,6f73b4d1ab8579ad,"[obtain, full, time, position, monday, friday, many, years, experience, looking, career, apply, skills, learn, new, things]","[great, experience, working, team, environment, well, alone, great, people, customer, service, skills, willingness, learn, new, things, hard, working, quick, learner, lots, experience, working, fa..."
10,c7675c0b793f34a0,"[looking, parttime, position, goal, focused, wellversed, restaurant, settings, drivin, go, beyond, satisfy, customers, improve, operations, support, team, members]","[skills, , high, energy, attitude, , sales, expertise, , team, leadership, , customer, relations, , complex, problem, solving, , time, management, , active, learning, , customer, support]"
11,35a0834424384bfa,[],"[skills, summary, highly, talented, cook, extensive, experience, preparing, ,, seasoning, cooking, wide, variety, meats, ,, fish, fowl, ,, vegetables, ,, soups, stocks, ,, breakfast, dishes, ,, pa..."
12,f6e0765504d64af7,"[, obtain, nuclear, operator, training, position, fulfill, passion, power, engineer]","[highlights, skills, /, knowledge, , proactive, attitude, towards, safety, ., , prioritizing, needs, resources, available, ., , always, giving, attention, every, team, members, opinion, resolve, s..."
